In [2]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0" bokeh==2.0.1
!pip install autogluon --no-cache-dir

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached setuptools-59.5.0-py3-none-any.whl (952 kB)
  Using cached wheel-0.37.0-py2.py3-none-any.whl (35 kB)
  Attempting uninstall: wheel
    Found existing installation: wheel

In [3]:
import pandas as pd
from autogluon.tabular import TabularPredictor

In [26]:
train = pd.read_csv("./train.csv")
train['datetime'] = pd.to_datetime(train['datetime'])
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [27]:
test = pd.read_csv("./test.csv")
test['datetime'] = pd.to_datetime(test['datetime'])
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [28]:
submission =  pd.read_csv("./sampleSubmission.csv")
submission['datetime'] = pd.to_datetime(submission['datetime'])
submission.head()

,datetime,count
0,2011-01-20 00:00:00,0
1,2011-01-20 01:00:00,0
2,2011-01-20 02:00:00,0
3,2011-01-20 03:00:00,0
4,2011-01-20 04:00:00,0


In [29]:
train["year"] = train.datetime.dt.year
train["month"] = train.datetime.dt.month
train["day"] = train.datetime.dt.day
train["hour"] = train.datetime.dt.hour
test["year"] = test.datetime.dt.year
test["month"] = test.datetime.dt.month
test["day"] = test.datetime.dt.day
test["hour"] = test.datetime.dt.hour


In [30]:
train["season"] = train.season.astype('category')
train["weather"] = train.weather.astype('category')
test["season"] = test.season.astype('category')
test["weather"] = test.weather.astype('category')

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  category      
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  category      
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
 12  year        10886 non-null  int64         
 13  month       10886 non-null  int64         
 14  day         10886 non-null  int64         
 15  hour        10886 non-null  int64         
dtypes: category(2), dateti

In [31]:
predictor_new_features = TabularPredictor(label="count", eval_metric="root_mean_squared_error", learner_kwargs={"ignored_columns": ["casual", "registered"]}).fit(train_data=train, time_limit=600, presets="best_quality")



No path specified. Models will be saved in: "AutogluonModels/ag-20211212_152012/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels/ag-20211212_152012/"
AutoGluon Version:  0.3.1
Train Data Rows:    10886
Train Data Columns: 15
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (977, 1, 191.57413, 181.14445)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Dropping user-specified ignored columns: ['casual', 'registered']
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1467.2 MB
	Train Data (Original)  Memory Usage: 0.98 MB (0.1% of availab

[1000]	train_set's rmse: 30.8186	valid_set's rmse: 38.4291
[2000]	train_set's rmse: 25.4239	valid_set's rmse: 36.1393
[3000]	train_set's rmse: 22.5686	valid_set's rmse: 35.6903
[4000]	train_set's rmse: 20.5877	valid_set's rmse: 35.5711
[5000]	train_set's rmse: 19.0435	valid_set's rmse: 35.5486


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 30.3019	valid_set's rmse: 41.0917
[2000]	train_set's rmse: 25.4089	valid_set's rmse: 38.7923
[3000]	train_set's rmse: 22.682	valid_set's rmse: 38.0445
[4000]	train_set's rmse: 20.7477	valid_set's rmse: 37.6701
[5000]	train_set's rmse: 19.2301	valid_set's rmse: 37.5143
[6000]	train_set's rmse: 18.0161	valid_set's rmse: 37.3559
[7000]	train_set's rmse: 16.9715	valid_set's rmse: 37.2906
[8000]	train_set's rmse: 16.0611	valid_set's rmse: 37.2206
[9000]	train_set's rmse: 15.2683	valid_set's rmse: 37.1952
[10000]	train_set's rmse: 14.5678	valid_set's rmse: 37.1924


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 21.0051	valid_set's rmse: 35.6024
[2000]	train_set's rmse: 15.4015	valid_set's rmse: 34.9824


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 21.3287	valid_set's rmse: 36.3245
[2000]	train_set's rmse: 15.6834	valid_set's rmse: 35.8282
[3000]	train_set's rmse: 12.2992	valid_set's rmse: 35.5694
[4000]	train_set's rmse: 10.1773	valid_set's rmse: 35.5318


	-36.5863	 = Validation score   (root_mean_squared_error)
	35.61s	 = Training   runtime
	1.49s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 248.32s of the 448.37s of remaining time.
	-41.3243	 = Validation score   (root_mean_squared_error)
	9.41s	 = Training   runtime
	0.4s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 236.06s of the 436.11s of remaining time.
	Time limit exceeded... Skipping CatBoost_BAG_L1.
Fitting model: ExtraTreesMSE_BAG_L1 ... Training model for up to 206.62s of the 406.67s of remaining time.
	-41.0311	 = Validation score   (root_mean_squared_error)
	3.81s	 = Training   runtime
	0.41s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ... Training model for up to 200.02s of the 400.07s of remaining time.
	Ran out of time, stopping training early. (Stopping on epoch 28)
	Ran out of time, stopping training early. (Stopping on epoch 28)
	Ran out of time, stopping training early. 

In [33]:
predictions_new_features = predictor_new_features.predict(test)
predictions_new_features[predictions_new_features < 0] = 0
submission_new_features = submission
submission_new_features["count"] = predictions_new_features
submission_new_features.to_csv("submission_new_features_1212.csv", index=False)
!kaggle competitions submit -c bike-sharing-demand -f submission_new_features_1212.csv -m "features original score"
!kaggle competitions submissions -c bike-sharing-demand | tail -n +1 | head -n 6

100%|█████████████████████████████████████████| 188k/188k [00:00<00:00, 471kB/s]
Successfully submitted to Bike Sharing DemandfileName                            date                 description                                     status    publicScore  privateScore  
----------------------------------  -------------------  ----------------------------------------------  --------  -----------  ------------  
submission_new_features_1212.csv    2021-12-12 15:31:23  features original score                         complete  0.47800      0.47800       
submission_fold5_levels1_extra.csv  2021-12-12 15:18:00  bag folds 5 stack levels 1 with extra features  complete  0.46747      0.46747       
submission_fold8_levels2.csv        2021-12-12 15:02:41  bag folds 8 stack levels 2                      complete  0.47674      0.47674       
submission_fold10_levels3.csv       2021-12-12 14:51:04  bag folds 10 stack levels 3                     complete  0.48236      0.48236       


In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  category      
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  category      
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
 12  year        10886 non-null  int64         
 13  month       10886 non-null  int64         
 14  day         10886 non-null  int64         
 15  hour        10886 non-null  int64         
dtypes: category(2), dateti

In [14]:
predictor_new_hpo = TabularPredictor(label="count", eval_metric="root_mean_squared_error", learner_kwargs={"ignored_columns": ["casual", "registered"]}).fit(train_data=train, time_limit=600, presets="best_quality", hyperparameter_tune_kwargs='auto', num_bag_folds=5, num_stack_levels=1)


No path specified. Models will be saved in: "AutogluonModels/ag-20211212_142649/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels/ag-20211212_142649/"
AutoGluon Version:  0.3.1
Train Data Rows:    10886
Train Data Columns: 15
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (977, 1, 191.57413, 181.14445)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Dropping user-specified ignored columns: ['casual', 'registered']
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    3076.0 MB
	Train Data (Original)  Memory Usage: 0.98 MB (0.0% of availab

[1000]	train_set's rmse: 30.3998	valid_set's rmse: 40.9151
[2000]	train_set's rmse: 24.9591	valid_set's rmse: 38.5055
[3000]	train_set's rmse: 22.0783	valid_set's rmse: 37.9068
[4000]	train_set's rmse: 20.0165	valid_set's rmse: 37.6446


	Ran out of time, early stopping on iteration 4720. Best iteration is:
	[4687]	train_set's rmse: 18.9103	valid_set's rmse: 37.5296
	Time limit exceeded
Fitted model: LightGBMXT_BAG_L1/T0 ...
	-37.5296	 = Validation score   (root_mean_squared_error)
	5.79s	 = Training   runtime
	0.72s	 = Validation runtime
Hyperparameter tuning model: LightGBM_BAG_L1 ...
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 20.8534	valid_set's rmse: 36.7752
[2000]	train_set's rmse: 15.0094	valid_set's rmse: 36.4139
[3000]	train_set's rmse: 11.442	valid_set's rmse: 36.3085


	Time limit exceeded
Fitted model: LightGBM_BAG_L1/T0 ...
	-36.2881	 = Validation score   (root_mean_squared_error)
	4.19s	 = Training   runtime
	0.41s	 = Validation runtime
Hyperparameter tuning model: RandomForestMSE_BAG_L1 ...
Fitted model: RandomForestMSE_BAG_L1/T0 ...
	-41.3243	 = Validation score   (root_mean_squared_error)
	9.55s	 = Training   runtime
	0.4s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1 ...
	Time limit exceeded
Fitted model: CatBoost_BAG_L1/T0 ...
	-39.3393	 = Validation score   (root_mean_squared_error)
	6.47s	 = Training   runtime
	0.01s	 = Validation runtime
Hyperparameter tuning model: ExtraTreesMSE_BAG_L1 ...
Fitted model: ExtraTreesMSE_BAG_L1/T0 ...
	-41.0311	 = Validation score   (root_mean_squared_error)
	3.91s	 = Training   runtime
	0.39s	 = Validation runtime
Hyperparameter tuning model: NeuralNetFastAI_BAG_L1 ...
	Ran out of time, stopping training early. (Stopping on epoch 18)
Fitted model: NeuralNetFastAI_BAG_L1/T0 ...
	-99.1334	

...

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 20.4609	valid_set's rmse: 37.3034


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 20.7576	valid_set's rmse: 39.2821
[2000]	train_set's rmse: 14.8007	valid_set's rmse: 38.9535
[3000]	train_set's rmse: 11.4219	valid_set's rmse: 38.9237


	-37.011	 = Validation score   (root_mean_squared_error)
	14.89s	 = Training   runtime
	2.39s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1/T0 ... Training model for up to 271.26s of the 471.3s of remaining time.
	-41.3243	 = Validation score   (root_mean_squared_error)
	9.55s	 = Training   runtime
	0.4s	 = Validation runtime
Fitting model: CatBoost_BAG_L1/T0 ... Training model for up to 271.19s of the 471.23s of remaining time.
	-37.1023	 = Validation score   (root_mean_squared_error)
	166.95s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1/T0 ... Training model for up to 110.05s of the 310.09s of remaining time.
	-41.0311	 = Validation score   (root_mean_squared_error)
	3.91s	 = Training   runtime
	0.39s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1/T0 ... Training model for up to 109.98s of the 310.02s of remaining time.
	-60.9513	 = Validation score   (root_mean_squared_error)
	44.54s	 = Training   runtime
	0.2s	 = 

In [16]:
predictions_new_hpo = predictor_new_hpo.predict(test)
predictions_new_hpo[predictions_new_hpo < 0] = 0
submission_new_hpo = submission
submission_new_hpo["count"] = predictions_new_hpo
submission_new_hpo.to_csv("submission_fold5_levels1.csv", index=False)
!kaggle competitions submit -c bike-sharing-demand -f submission_fold5_levels1.csv -m "bag folds 5 stack levels 1"
!kaggle competitions submissions -c bike-sharing-demand | tail -n +1 | head -n 6

100%|█████████████████████████████████████████| 188k/188k [00:00<00:00, 597kB/s]
Successfully submitted to Bike Sharing DemandfileName                      date                 description                        status    publicScore  privateScore  
----------------------------  -------------------  ---------------------------------  --------  -----------  ------------  
submission_fold5_levels1.csv  2021-12-12 14:38:11  bag folds 5 stack levels 1         complete  0.46811      0.46811       
submission_final.csv          2021-12-12 11:33:27  new features with hpo              complete  0.47905      0.47905       
submission_extra_time.csv     2021-12-12 10:42:44  extra time                         complete  0.56132      0.56132       
submission_extra_time.csv     2021-12-12 10:42:37  extra time                         complete  0.56132      0.56132       


In [17]:
predictor_new_hpo = TabularPredictor(label="count", eval_metric="root_mean_squared_error", learner_kwargs={"ignored_columns": ["casual", "registered"]}).fit(train_data=train, time_limit=600, presets="best_quality", hyperparameter_tune_kwargs='auto', num_bag_folds=10, num_stack_levels=3)

No path specified. Models will be saved in: "AutogluonModels/ag-20211212_143950/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels/ag-20211212_143950/"
AutoGluon Version:  0.3.1
Train Data Rows:    10886
Train Data Columns: 15
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (977, 1, 191.57413, 181.14445)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Dropping user-specified ignored columns: ['casual', 'registered']
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1939.27 MB
	Train Data (Original)  Memory Usage: 0.98 MB (0.1% of availa

[1000]	train_set's rmse: 30.3019	valid_set's rmse: 41.0917
[2000]	train_set's rmse: 25.4089	valid_set's rmse: 38.7923
[3000]	train_set's rmse: 22.682	valid_set's rmse: 38.0445
[4000]	train_set's rmse: 20.7477	valid_set's rmse: 37.6701
[5000]	train_set's rmse: 19.2301	valid_set's rmse: 37.5143
[6000]	train_set's rmse: 18.0161	valid_set's rmse: 37.3559
[7000]	train_set's rmse: 16.9715	valid_set's rmse: 37.2906
[8000]	train_set's rmse: 16.0611	valid_set's rmse: 37.2206
[9000]	train_set's rmse: 15.2683	valid_set's rmse: 37.1952
[10000]	train_set's rmse: 14.5678	valid_set's rmse: 37.1924


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 30.8968	valid_set's rmse: 35.5721
[2000]	train_set's rmse: 25.8631	valid_set's rmse: 34.288
[3000]	train_set's rmse: 23.0637	valid_set's rmse: 34.1613


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 31.0398	valid_set's rmse: 37.6854
[2000]	train_set's rmse: 25.8317	valid_set's rmse: 35.4668
[3000]	train_set's rmse: 23.0887	valid_set's rmse: 34.8136
[4000]	train_set's rmse: 21.1229	valid_set's rmse: 34.5371
[5000]	train_set's rmse: 19.5767	valid_set's rmse: 34.3685
[6000]	train_set's rmse: 18.3516	valid_set's rmse: 34.2276
[7000]	train_set's rmse: 17.2755	valid_set's rmse: 34.2726


	-37.0935	 = Validation score   (root_mean_squared_error)
	87.37s	 = Training   runtime
	4.51s	 = Validation runtime
Fitting model: LightGBM_BAG_L1/T0 ... Training model for up to 55.94s of the 455.88s of remaining time.
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 21.0431	valid_set's rmse: 36.9388
[2000]	train_set's rmse: 15.5393	valid_set's rmse: 36.1223
[3000]	train_set's rmse: 12.2068	valid_set's rmse: 35.9677
[4000]	train_set's rmse: 9.97753	valid_set's rmse: 35.9639


	Ran out of time, early stopping on iteration 4219. Best iteration is:
	[3581]	train_set's rmse: 10.8409	valid_set's rmse: 35.9108
	Time limit exceeded... Skipping LightGBM_BAG_L1/T0.
Fitting model: RandomForestMSE_BAG_L1/T0 ... Training model for up to 49.21s of the 449.15s of remaining time.
	-41.3243	 = Validation score   (root_mean_squared_error)
	9.51s	 = Training   runtime
	0.4s	 = Validation runtime
Fitting model: CatBoost_BAG_L1/T0 ... Training model for up to 49.12s of the 449.07s of remaining time.
	Time limit exceeded... Skipping CatBoost_BAG_L1/T0.
Fitting model: ExtraTreesMSE_BAG_L1/T0 ... Training model for up to 41.8s of the 441.74s of remaining time.
	-41.0311	 = Validation score   (root_mean_squared_error)
	3.8s	 = Training   runtime
	0.39s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1/T0 ... Training model for up to 41.73s of the 441.67s of remaining time.
	Ran out of time, stopping training early. (Stopping on epoch 5)
	Ran out of time, stopping trainin

In [18]:
predictions_new_hpo = predictor_new_hpo.predict(test)
predictions_new_hpo[predictions_new_hpo < 0] = 0
submission_new_hpo = submission
submission_new_hpo["count"] = predictions_new_hpo
submission_new_hpo.to_csv("submission_fold10_levels3.csv", index=False)
!kaggle competitions submit -c bike-sharing-demand -f submission_fold10_levels3.csv -m "bag folds 10 stack levels 3"
!kaggle competitions submissions -c bike-sharing-demand | tail -n +1 | head -n 6

100%|█████████████████████████████████████████| 188k/188k [00:00<00:00, 436kB/s]
Successfully submitted to Bike Sharing DemandfileName                       date                 description                        status    publicScore  privateScore  
-----------------------------  -------------------  ---------------------------------  --------  -----------  ------------  
submission_fold10_levels3.csv  2021-12-12 14:51:04  bag folds 10 stack levels 3        complete  0.48236      0.48236       
submission_fold5_levels1.csv   2021-12-12 14:38:11  bag folds 5 stack levels 1         complete  0.46811      0.46811       
submission_final.csv           2021-12-12 11:33:27  new features with hpo              complete  0.47905      0.47905       
submission_extra_time.csv      2021-12-12 10:42:44  extra time                         complete  0.56132      0.56132       


In [20]:
predictor_new_hpo = TabularPredictor(label="count", eval_metric="root_mean_squared_error", learner_kwargs={"ignored_columns": ["casual", "registered"]}).fit(train_data=train, time_limit=600, presets="best_quality", hyperparameter_tune_kwargs='auto', num_bag_folds=8, num_stack_levels=2)

No path specified. Models will be saved in: "AutogluonModels/ag-20211212_145157/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels/ag-20211212_145157/"
AutoGluon Version:  0.3.1
Train Data Rows:    10886
Train Data Columns: 15
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (977, 1, 191.57413, 181.14445)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Dropping user-specified ignored columns: ['casual', 'registered']
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1571.62 MB
	Train Data (Original)  Memory Usage: 0.98 MB (0.1% of availa

[1000]	train_set's rmse: 30.5131	valid_set's rmse: 37.885


	Ran out of time, early stopping on iteration 1400. Best iteration is:
	[1400]	train_set's rmse: 28.0372	valid_set's rmse: 36.7886
	Time limit exceeded
Fitted model: LightGBMXT_BAG_L1/T0 ...
	-36.7886	 = Validation score   (root_mean_squared_error)
	2.39s	 = Training   runtime
	0.27s	 = Validation runtime
Hyperparameter tuning model: LightGBM_BAG_L1 ...
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 20.9168	valid_set's rmse: 35.483


	Time limit exceeded
Fitted model: LightGBM_BAG_L1/T0 ...
	-35.3473	 = Validation score   (root_mean_squared_error)
	2.19s	 = Training   runtime
	0.11s	 = Validation runtime
Hyperparameter tuning model: RandomForestMSE_BAG_L1 ...
Fitted model: RandomForestMSE_BAG_L1/T0 ...
	-41.3243	 = Validation score   (root_mean_squared_error)
	9.51s	 = Training   runtime
	0.4s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1 ...
	Time limit exceeded
Fitted model: CatBoost_BAG_L1/T0 ...
	-39.908	 = Validation score   (root_mean_squared_error)
	3.37s	 = Training   runtime
	0.01s	 = Validation runtime
Hyperparameter tuning model: ExtraTreesMSE_BAG_L1 ...
Fitted model: ExtraTreesMSE_BAG_L1/T0 ...
	-41.0311	 = Validation score   (root_mean_squared_error)
	3.8s	 = Training   runtime
	0.39s	 = Validation runtime
Hyperparameter tuning model: NeuralNetFastAI_BAG_L1 ...
	Ran out of time, stopping training early. (Stopping on epoch 3)
Fitted model: NeuralNetFastAI_BAG_L1/T0 ...
	-127.0717	 =

...

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 20.7115	valid_set's rmse: 40.4592
[2000]	train_set's rmse: 14.9862	valid_set's rmse: 39.9936


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 21.5082	valid_set's rmse: 36.9023
[2000]	train_set's rmse: 15.8354	valid_set's rmse: 36.3202
[3000]	train_set's rmse: 12.4293	valid_set's rmse: 36.2211
[4000]	train_set's rmse: 10.1025	valid_set's rmse: 36.2127


	-36.6135	 = Validation score   (root_mean_squared_error)
	34.66s	 = Training   runtime
	1.82s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1/T0 ... Training model for up to 90.55s of the 423.86s of remaining time.
	-41.3243	 = Validation score   (root_mean_squared_error)
	9.51s	 = Training   runtime
	0.4s	 = Validation runtime
Fitting model: CatBoost_BAG_L1/T0 ... Training model for up to 90.48s of the 423.79s of remaining time.
	Time limit exceeded... Skipping CatBoost_BAG_L1/T0.
Fitting model: ExtraTreesMSE_BAG_L1/T0 ... Training model for up to 73.28s of the 406.59s of remaining time.
	-41.0311	 = Validation score   (root_mean_squared_error)
	3.8s	 = Training   runtime
	0.39s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1/T0 ... Training model for up to 73.21s of the 406.52s of remaining time.
	Ran out of time, stopping training early. (Stopping on epoch 14)
	Ran out of time, stopping training early. (Stopping on epoch 14)
	Ran out of time, stopping traini

In [21]:
predictions_new_hpo = predictor_new_hpo.predict(test)
predictions_new_hpo[predictions_new_hpo < 0] = 0
submission_new_hpo = submission
submission_new_hpo["count"] = predictions_new_hpo
submission_new_hpo.to_csv("submission_fold8_levels2.csv", index=False)
!kaggle competitions submit -c bike-sharing-demand -f submission_fold8_levels2.csv -m "bag folds 8 stack levels 2"
!kaggle competitions submissions -c bike-sharing-demand | tail -n +1 | head -n 6

100%|█████████████████████████████████████████| 188k/188k [00:00<00:00, 565kB/s]
Successfully submitted to Bike Sharing DemandfileName                       date                 description                        status    publicScore  privateScore  
-----------------------------  -------------------  ---------------------------------  --------  -----------  ------------  
submission_fold8_levels2.csv   2021-12-12 15:02:41  bag folds 8 stack levels 2         complete  0.47674      0.47674       
submission_fold10_levels3.csv  2021-12-12 14:51:04  bag folds 10 stack levels 3        complete  0.48236      0.48236       
submission_fold5_levels1.csv   2021-12-12 14:38:11  bag folds 5 stack levels 1         complete  0.46811      0.46811       
submission_final.csv           2021-12-12 11:33:27  new features with hpo              complete  0.47905      0.47905       


In [22]:
def time_slot_cat(row):
    if row["hour"] >= 7 and row["hour"] <= 9:
        return "morning"
    elif row["hour"] >= 11 and row["hour"] <= 13:
        return "lunch"
    elif row["hour"] >= 17 and row["hour"] <= 18:
        return "evening"
    else:
         return "other"

def windy_cat(row):
    if row["windspeed"] < 20:
        return "calm"
    elif row["windspeed"] >= 20 and row["windspeed"] < 50:
        return "breeze"
    elif row["windspeed"] >= 50 and row["windspeed"] <= 88:
        return "windy"
    else:
        return "very windy"


def temperature_cat(row):
    if row["temp"] < 11:
        return "cold"
    elif row["temp"] >= 11 and row["temp"] <= 25:
        return "mild"
    else:
        return "hot" 

train["time_slot"] = train.apply(time_slot_cat, axis=1)
train["time_slot"] = train["time_slot"].astype("category")
test["time_slot"] = test.apply(time_slot_cat, axis=1)
test["time_slot"] = test["time_slot"].astype("category")


train["wind_type"] = train.apply(windy_cat, axis=1)
train["wind_type"] = train["wind_type"].astype("category")
test["wind_type"] = test.apply(windy_cat, axis=1)
test["wind_type"] = test["wind_type"].astype("category")


train["temp_type"] = train.apply(temperature_cat, axis=1)
train["temp_type"] = train["temp_type"].astype("category")
test["temp_type"] = test.apply(temperature_cat, axis=1)
test["temp_type"] = test["temp_type"].astype("category")



In [ ]:
predictor_new_hpo = TabularPredictor(label="count", eval_metric="root_mean_squared_error", learner_kwargs={"ignored_columns": ["casual", "registered"]}).fit(train_data=train, time_limit=600, presets="best_quality", hyperparameter_tune_kwargs='auto', num_bag_folds=5, num_stack_levels=1)

No path specified. Models will be saved in: "AutogluonModels/ag-20211212_150708/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels/ag-20211212_150708/"
AutoGluon Version:  0.3.1
Train Data Rows:    10886
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (977, 1, 191.57413, 181.14445)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Dropping user-specified ignored columns: ['casual', 'registered']
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1634.22 MB
	Train Data (Original)  Memory Usage: 1.01 MB (0.1% of availa

[1000]	train_set's rmse: 28.7647	valid_set's rmse: 39.9449
[2000]	train_set's rmse: 23.6093	valid_set's rmse: 38.7992
[3000]	train_set's rmse: 20.5876	valid_set's rmse: 38.4824


	Ran out of time, early stopping on iteration 3746. Best iteration is:
	[3660]	train_set's rmse: 19.123	valid_set's rmse: 38.3059
	Time limit exceeded
Fitted model: LightGBMXT_BAG_L1/T0 ...
	-38.3059	 = Validation score   (root_mean_squared_error)
	5.89s	 = Training   runtime
	0.64s	 = Validation runtime
Hyperparameter tuning model: LightGBM_BAG_L1 ...
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 19.8977	valid_set's rmse: 37.159
[2000]	train_set's rmse: 14.0235	valid_set's rmse: 36.58
[3000]	train_set's rmse: 10.91	valid_set's rmse: 36.4851


	Time limit exceeded
Fitted model: LightGBM_BAG_L1/T0 ...
	-36.4577	 = Validation score   (root_mean_squared_error)
	4.93s	 = Training   runtime
	0.38s	 = Validation runtime
Hyperparameter tuning model: RandomForestMSE_BAG_L1 ...
Fitted model: RandomForestMSE_BAG_L1/T0 ...
	-41.7202	 = Validation score   (root_mean_squared_error)
	10.74s	 = Training   runtime
	0.4s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1 ...
	Time limit exceeded
Fitted model: CatBoost_BAG_L1/T0 ...
	-39.9877	 = Validation score   (root_mean_squared_error)
	10.49s	 = Training   runtime
	0.01s	 = Validation runtime
Hyperparameter tuning model: ExtraTreesMSE_BAG_L1 ...
Fitted model: ExtraTreesMSE_BAG_L1/T0 ...
	-41.308	 = Validation score   (root_mean_squared_error)
	4.32s	 = Training   runtime
	0.4s	 = Validation runtime
Hyperparameter tuning model: NeuralNetFastAI_BAG_L1 ...
	Ran out of time, stopping training early. (Stopping on epoch 9)
Fitted model: NeuralNetFastAI_BAG_L1/T0 ...
	-129.3432	

...

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 19.7383	valid_set's rmse: 37.756
[2000]	train_set's rmse: 13.8378	valid_set's rmse: 37.3779


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 20.1434	valid_set's rmse: 39.0001
[2000]	train_set's rmse: 14.2062	valid_set's rmse: 38.714


	-37.0062	 = Validation score   (root_mean_squared_error)
	21.91s	 = Training   runtime
	1.76s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1/T0 ... Training model for up to 260.36s of the 460.4s of remaining time.
	-41.7202	 = Validation score   (root_mean_squared_error)
	10.74s	 = Training   runtime
	0.4s	 = Validation runtime
Fitting model: CatBoost_BAG_L1/T0 ... Training model for up to 260.28s of the 460.32s of remaining time.
	-37.7988	 = Validation score   (root_mean_squared_error)
	244.75s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1/T0 ... Training model for up to 25.42s of the 225.45s of remaining time.
	-41.308	 = Validation score   (root_mean_squared_error)
	4.32s	 = Training   runtime
	0.4s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1/T0 ... Training model for up to 25.34s of the 225.38s of remaining time.
	Ran out of time, stopping training early. (Stopping on epoch 8)
	Ran out of time, stopping trainin

In [ ]:
predictions_new_hpo = predictor_new_hpo.predict(test)
predictions_new_hpo[predictions_new_hpo < 0] = 0
submission_new_hpo = submission
submission_new_hpo["count"] = predictions_new_hpo
submission_new_hpo.to_csv("submission_fold5_levels1_extra.csv", index=False)
!kaggle competitions submit -c bike-sharing-demand -f submission_fold5_levels1_extra.csv -m "bag folds 5 stack levels 1 with extra features"
!kaggle competitions submissions -c bike-sharing-demand | tail -n +1 | head -n 6